In [1]:
import deep_image_matching as dim

params = {
    "dir": "./assets/example_cyprus",
    "pipeline": "superpoint+lightglue",
    "strategy": "matching_lowres",
    "quality": "medium",
    "tiling": "none",
    "camera_options": "./assets/example_cyprus/cameras.yaml",
    "openmvg": None,
}
config = dim.Config(params)

Deep Image Matching loaded in 3.001 seconds.


Run feature extraction and matching

In [2]:
matcher = dim.ImageMatcher(config)
feature_path, match_path = matcher.run()

Loaded SuperPoint model
2024-04-12 15:35:20 | [INFO    ] Running image matching with the following configuration:
2024-04-12 15:35:20 | [INFO    ]   Image folder: assets/example_cyprus/images
2024-04-12 15:35:20 | [INFO    ]   Output folder: assets/example_cyprus/results_superpoint+lightglue_matching_lowres_quality_medium
2024-04-12 15:35:20 | [INFO    ]   Number of images: 10
2024-04-12 15:35:20 | [INFO    ]   Matching strategy: matching_lowres
2024-04-12 15:35:20 | [INFO    ]   Image quality: MEDIUM
2024-04-12 15:35:20 | [INFO    ]   Tile selection: NONE
2024-04-12 15:35:20 | [INFO    ]   Feature extraction method: superpoint
2024-04-12 15:35:20 | [INFO    ]   Matching method: lightglue
2024-04-12 15:35:20 | [INFO    ]   Geometric verification: PYDEGENSAC
2024-04-12 15:35:20 | [INFO    ]   CUDA available: True
2024-04-12 15:35:20 | [INFO    ] Low resolution matching, generating pairs ..
Loaded SuperPoint model
2024-04-12 15:35:20 | [INFO    ] Extracting features from downsampled imag

100%|██████████| 10/10 [00:02<00:00,  3.92it/s]

2024-04-12 15:35:23 | [INFO    ] Matching downsampled images...



100%|██████████| 45/45 [00:00<00:00, 46.17it/s]

2024-04-12 15:35:24 | [INFO    ] Found 28 pairs.
2024-04-12 15:35:24 | [INFO    ] Extracting features with superpoint...
2024-04-12 15:35:24 | [INFO    ] superpoint configuration: 


{'keypoint_threshold': 0.0005,
 'max_keypoints': 4096,
 'name': 'superpoint',
 'nms_radius': 3}


100%|██████████| 10/10 [00:00<00:00, 11.39it/s]

2024-04-12 15:35:25 | [INFO    ] Features extracted!
2024-04-12 15:35:25 | [INFO    ] Matching features with lightglue...
2024-04-12 15:35:25 | [INFO    ] lightglue configuration: 
{'depth_confidence': 0.95,
 'filter_threshold': 0.1,
 'flash': True,
 'mp': False,
 'n_layers': 9,
 'name': 'lightglue',
 'width_confidence': 0.99}
2024-04-12 15:35:25 | [INFO    ] Matching features...
2024-04-12 15:35:25 | [INFO    ] 



100%|██████████| 28/28 [00:08<00:00,  3.34it/s]

2024-04-12 15:35:33 | [INFO    ] [Timer] | [matching] generate_pairs=7.545, extract_features=0.887, Match pair=0.367, Total execution=16.829
2024-04-12 15:35:33 | [INFO    ] [Timer] | [Deep Image Matching] Total execution=0.000


Export matches to COLMAP format

In [3]:
dim.io.export_to_colmap(
    img_dir=config.general["image_dir"],
    feature_path=feature_path,
    match_path=match_path,
    database_path=config.general["output_dir"] / "database.db",
    camera_config_path=config.general["camera_options"],
)

2024-04-12 15:35:37 | [WARNING ] Was not possible to load the first image to initialize cam0
2024-04-12 15:35:37 | [WARNING ] Was not possible to load the first image to initialize cam1


100%|██████████| 10/10 [00:00<00:00, 1252.93it/s]
28it [00:00, 7109.85it/s]             
28it [00:00, 8581.70it/s]             


Run reconstruction with pycolmap

In [4]:
model = dim.reconstruction.pycolmap_reconstruction(
    database_path=config.general["output_dir"] / "database.db",
    sfm_dir=config.general["output_dir"],
    image_dir=config.general["image_dir"],
    refine_intrinsics=False,
)

2024-04-12 15:35:41 | [INFO    ] Running 3D reconstruction...


I20240412 15:35:41.812327 3256596 misc.cc:198] 
Loading database
I20240412 15:35:41.813288 3256596 database_cache.cc:54] Loading cameras...
I20240412 15:35:41.813314 3256596 database_cache.cc:64]  1 in 0.000s
I20240412 15:35:41.813323 3256596 database_cache.cc:72] Loading matches...
I20240412 15:35:41.813802 3256596 database_cache.cc:78]  28 in 0.000s
I20240412 15:35:41.813812 3256596 database_cache.cc:94] Loading images...
I20240412 15:35:41.814779 3256596 database_cache.cc:143]  10 in 0.001s (connected 8)
I20240412 15:35:41.814792 3256596 database_cache.cc:154] Building correspondence graph...
I20240412 15:35:41.819002 3256596 database_cache.cc:190]  in 0.004s (ignored 0)
I20240412 15:35:41.819049 3256596 timer.cc:91] Elapsed time: 0.000 [minutes]
I20240412 15:35:41.819574 3256596 misc.cc:198] 
Finding good initial image pair
I20240412 15:35:41.969928 3256596 misc.cc:198] 
Initializing with image pair #5 and #9
I20240412 15:35:41.973089 3256596 misc.cc:198] 
Global bundle adjustment


2024-04-12 15:35:46 | [INFO    ] Reconstructed 1 model(s).
2024-04-12 15:35:46 | [INFO    ] Largest model is #0 with 8 images.
2024-04-12 15:35:46 | [INFO    ] Exporting model...


I20240412 15:35:46.915881 3256596 timer.cc:91] Elapsed time: 0.085 [minutes]
